# Data exploration

In [1]:
import os
import zipfile
import requests

def download_data(url, name, path='data'):
    if not os.path.exists(path):
        os.mkdir(path)

    response = requests.get(url)
    with open(os.path.join(path, name), 'wb') as f:
        f.write(response.content)

    z = zipfile.ZipFile(os.path.join(path, 'vehicles.zip'))
    z.extractall(path)

VEHICLES = 'http://bit.ly/ddl-cars'

download_data(VEHICLES, 'vehicles.zip')

In [2]:
# load it into a pandas data frame.
import pandas as pd
path = 'data'
vehicles = pd.read_csv(os.path.join(path, 'vehicles.csv'))

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [6]:
vehicles.describe()

,year,displ,cylinders,barrels08,city08,highway08,comb08,co2TailpipeGpm,fuelCost08
count,35095.000000,35095.000000,35095.000000,35095.000000,35095.000000,35095.000000,35095.000000,35095.000000,35095.000000
mean,2000.318763,3.343015,5.766747,17.670948,17.580453,23.788574,19.853255,477.000685,1897.275965
std,9.877432,1.358464,1.749489,4.468803,4.725980,5.864402,5.075383,119.061076,507.200520
min,1984.000000,0.600000,2.000000,0.060000,6.000000,9.000000,7.000000,37.000000,600.000000
25%,1991.000000,2.200000,4.000000,14.982273,15.000000,20.000000,16.000000,403.954545,1500.000000
50%,2001.000000,3.000000,6.000000,17.347895,17.000000,24.000000,19.000000,467.736842,1850.000000
75%,2009.000000,4.300000,6.000000,20.600625,20.000000,27.000000,22.000000,555.437500,2200.000000
max,2016.000000,8.400000,16.000000,47.087143,58.000000,61.000000,56.000000,1269.571429,5800.000000


Clean it up by dropping columns we don't need, removing vehicles that are coming out in the future, removing any duplicate records, and then sorting the data by make, model, and year.

In [7]:
select_columns = ['make', 'model', 'year', 'displ', 'cylinders', 'trany', 'drive', 'VClass','fuelType','barrels08', 'city08', 'highway08', 'comb08', 'co2TailpipeGpm', 'fuelCost08']

vehicles = vehicles[select_columns][vehicles.year <= 2016].drop_duplicates().dropna()

vehicles = vehicles.sort(['make', 'model', 'year'])

vehicles.head()

,make,model,year,displ,cylinders,trany,drive,VClass,fuelType,barrels08,city08,highway08,comb08,co2TailpipeGpm,fuelCost08
19316,AM General,DJ Po Vehicle 2WD,1984,2.5,4,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,19.388824,18,17,17,522.764706,1950
19314,AM General,FJ8c Post Office,1984,4.2,6,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
358,AM General,Post Office DJ5 2WD,1985,2.5,4,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,20.600625,16,17,16,555.437500,2100
369,AM General,Post Office DJ8 2WD,1985,4.2,6,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,25.354615,13,13,13,683.615385,2550
25797,ASC Incorporated,GNX,1987,3.8,6,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,20.600625,14,21,16,555.437500,2550


## Identify stage

**Prep Phase**. The first thing we're going to do is identify the types of information contained in our data set, which will help us get to know our data a bit better and prepare us to think about the data in different ways. After that, we will identify the **entities** in our data set so that we are aware of the different levels to which we can **aggregate** up or **drill down**.

## Types of Information

There are a few distinct types of information that jump out at us just from taking a quick look at the data set.

- Vehicle attributes information
- Vehicle manufacturer information
- Engine information
- Fuel information (such as fuel efficiency, fuel type, and fuel cost)
- Transmission information
- Drive axle information

